In [2]:
#Importing Libraries

import pandas as pd
import numpy 
import json
import csv
import requests
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# Reading the file

df = pd.read_csv('animes.csv', encoding="unicode_escape", error_bad_lines=False)

In [4]:
df

,anime_id,title,genre,aired,episodes,members,score
0,1,Zombieland Saga: Revenge,"['Music', 'Comedy', 'Supernatural']",2020,12,168000,8.04
1,2,Yuukoku no Moriarty,"['Mystery', 'Historical', 'Psychological', 'Sh...",2020,12,297935,8.13
2,3,Yesterday wo Utatte,"['Slice of Life', 'Drama', 'Romance', 'Seinen']",2020,12,263517,6.91
3,4,Yes ka No ka Hanbun ka,"['Drama', 'Shounen Ai']",2020,1,38555,7.12
4,5,Yahari Ore no Seishun Love Comedy wa Machigatt...,"['Slice of Life', 'Comedy', 'Drama', 'Romance'...",2020,12,572517,8.39
5,6,Wu Shan Wu Xing,"['Action', 'Fantasy', 'Historical', 'Martial A...",2020,3,38966,7.91
6,7,World Trigger 2,"['Action', 'Sci-Fi', 'Supernatural', 'School',...",2020,12,109697,8.09
7,8,Violet Evergarden Movie,"['Drama', 'Fantasy', 'Slice of Life']",2020,1,397710,8.96
8,9,Umibe no EÂtranger,"['Slice of Life', 'Romance', 'Shounen Ai']",2020,1,117000,7.94
9,10,Toaru Kagaku no Railgun T,"['Action', 'Sci-Fi', 'Super Power']",2020,25,188839,8.20


In [5]:
features = ["title", "genre", "aired", "episodes"]

In [6]:
df.shape[0]

2866

In [7]:
#Combining features that are factors for choosing an anime
def combine_features(data):
    
    features = []
    
    for i in range(0, data.shape[0]):
        features.append(data["title"][i]+' '+data['genre'][i]+' '+str(data['aired'][i])+' '+str(data['episodes'][i]))
                        
    return features

#data["title"][i]+' '+data['genre'][i]+' '+str(data['aired'][i])+' '+str(data['episodes'][i])

In [8]:
#Create a column for all the combined features
df['features'] = combine_features(df)
#Show data
df["features"][0]

"Zombieland Saga: Revenge ['Music', 'Comedy', 'Supernatural'] 2020 12"

In [9]:
df

,anime_id,title,genre,aired,episodes,members,score,features
0,1,Zombieland Saga: Revenge,"['Music', 'Comedy', 'Supernatural']",2020,12,168000,8.04,"Zombieland Saga: Revenge ['Music', 'Comedy', '..."
1,2,Yuukoku no Moriarty,"['Mystery', 'Historical', 'Psychological', 'Sh...",2020,12,297935,8.13,"Yuukoku no Moriarty ['Mystery', 'Historical', ..."
2,3,Yesterday wo Utatte,"['Slice of Life', 'Drama', 'Romance', 'Seinen']",2020,12,263517,6.91,"Yesterday wo Utatte ['Slice of Life', 'Drama',..."
3,4,Yes ka No ka Hanbun ka,"['Drama', 'Shounen Ai']",2020,1,38555,7.12,"Yes ka No ka Hanbun ka ['Drama', 'Shounen Ai']..."
4,5,Yahari Ore no Seishun Love Comedy wa Machigatt...,"['Slice of Life', 'Comedy', 'Drama', 'Romance'...",2020,12,572517,8.39,Yahari Ore no Seishun Love Comedy wa Machigatt...
5,6,Wu Shan Wu Xing,"['Action', 'Fantasy', 'Historical', 'Martial A...",2020,3,38966,7.91,"Wu Shan Wu Xing ['Action', 'Fantasy', 'Histori..."
6,7,World Trigger 2,"['Action', 'Sci-Fi', 'Supernatural', 'School',...",2020,12,109697,8.09,"World Trigger 2 ['Action', 'Sci-Fi', 'Supernat..."
7,8,Violet Evergarden Movie,"['Drama', 'Fantasy', 'Slice of Life']",2020,1,397710,8.96,"Violet Evergarden Movie ['Drama', 'Fantasy', '..."
8,9,Umibe no EÂtranger,"['Slice of Life', 'Romance', 'Shounen Ai']",2020,1,117000,7.94,"Umibe no EÂtranger ['Slice of Life', 'Romance..."
9,10,Toaru Kagaku no Railgun T,"['Action', 'Sci-Fi', 'Super Power']",2020,25,188839,8.20,"Toaru Kagaku no Railgun T ['Action', 'Sci-Fi',..."


In [10]:
#Convert the text from the new column to a matrix of word counts:
cm = CountVectorizer(stop_words='english').fit_transform(df['features'])

In [11]:
#Get Cosine Similarity Matrix from Count Matrix

cs = cosine_similarity(cm, cm)

#print scores

print(cs)

/Users/saisanthoshthunga/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


[[1.         0.25       0.2236068  ... 0.23570226 0.         0.09128709]
 [0.25       1.         0.2236068  ... 0.23570226 0.         0.        ]
 [0.2236068  0.2236068  1.         ... 0.31622777 0.         0.32659863]
 ...
 [0.23570226 0.23570226 0.31622777 ... 1.         0.         0.25819889]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.09128709 0.         0.32659863 ... 0.25819889 0.         1.        ]]


In [12]:
#Get the title of the anime the viewer likes
Title = df["title"][909]
Title

'Kimetsu no Yaiba'

In [13]:
#Get the anime_id
anime_id = df[df.title == Title]["anime_id"].values[0]
#Show anime_id
anime_id

910

In [14]:
#Create a list of tuples in the form (anime_id, similarity score)
scores = list(enumerate(cs[anime_id]))
print(scores[0:6])

[(0, 0.24999999999999994), (1, 0.12499999999999997), (2, 0.0), (3, 0.0), (4, 0.16666666666666666), (5, 0.0)]


In [15]:
sorted_scores = sorted(scores, key=lambda x:x[1], reverse=True)
sorted_scores = sorted_scores[1:]
sorted_scores[:10]

[(544, 0.6681531047810608),
 (592, 0.6681531047810608),
 (100, 0.533001790889026),
 (638, 0.533001790889026),
 (2684, 0.533001790889026),
 (1229, 0.4999999999999999),
 (217, 0.47434164902525683),
 (2414, 0.47434164902525683),
 (906, 0.4714045207910316),
 (800, 0.4472135954999579)]

In [16]:
#print the top 10 recommended animes

j = 0

print('Top 10 Recommended Animes ' + Title + ' are:\n:')
for item in sorted_scores:
    anime_title = df[df.anime_id == item[0]]['title'].values[0]
    print(j+1, anime_title)
    j = j+1
    
    if j >= 10:
        break

Top 10 Recommended Animes Kimetsu no Yaiba are:
:
1 Bakuman. 2nd Season
2 Rainbow: Nisha Rokubou no Shichinin
3 Magic Kaito
4 Mushishi Zoku Shou 2nd Season
5 Strawberry Panic
6 Monochrome Factor
7 Kamisama Hajimemashita OVA
8 Beatless
9 Fullmetal Alchemist: Brotherhood
10 Aoi Bungaku Series
